In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import utilities.plotting_utilities
import pandas as pd

from initial_states.parallelclifford import ParallelCliffordLoader
from initial_states.pyramidclifford import PyramidCliffordLoader

### Validation of the DPP process sampled with the quantum circuit

Random matrix with normalized columns

In [2]:
n = 16
rank = 3
np.random.seed(0) # fixing the seed for reproducibility
mat = np.random.randn(n,rank)
Q = mat / np.sqrt(np.sum(mat*mat, axis=0)) 
# Q, _ = np.linalg.qr(np.random.randn(n, n)) # for std projection DPP # K = W @ W.T # the DPP kernel # only when Q comes from QR
W = Q[:,:rank]

Implementing the corresponding Clifford loader


In [ ]:
circuit = ParallelCliffordLoader(W) # (with FBS gates)
# circuit = PyramidCliffordLoader(W) # (with only RBS gates)
# Add measurements
meas = qk.QuantumCircuit(n, n) # n qubits, n classical bits
meas.barrier(range(n)) # the barrier is optional, it is an instruction for the later transpiler
meas.measure(range(n), range(n)) # perform the measurement, record it in the classical bits
circuit.add_register(meas.cregs[0])
qc = circuit.compose(meas)
# drawing
qc.decompose().draw(
    output="mpl", 
    filename="fig/example_circuit.pdf",
    scale = 1,
    fold=300)

Sampling from the circuit

In [8]:
# Import Aer and set the backend simulator
from qiskit_aer import Aer
# can also select largest sample cardinal to be examined
max_cardinality = 2 # only compare probabilities on subset of cardinal 2
# max_cardinality = 1 # only compare probabilities on subset of cardinal 2
backend_sim = Aer.get_backend('qasm_simulator')
# Execute the circuit on the qasm simulator, we do several repeats to get 
# the distribution of the TV between the empirical measure and the underlying DPP.
tv_distances = []
num_repeats = 100
num_shots = 20000
proba = {} # add empty set for compatibility with empty IBM measurements
proba_estimated = {}
for _ in range(num_repeats):
    job_sim = backend_sim.run(
        qk.transpile(qc, backend_sim), 
        shots=num_shots
    )
    result_sim = job_sim.result()
    counts = result_sim.get_counts(qc)
    proba, proba_estimated = utilities.plotting_utilities.get_estimated_probabilities_skew_cutoff(W,max_cardinality, counts, num_shots)
    tv_distances.append(
        0.5*np.sum([np.abs(proba[key]-proba_estimated[key]) for key in proba])
    )

Histogram of the tv distances between empirical and exact probabilities

In [ ]:
# obtained by using a cutoff on sample size
plt.hist(tv_distances)
plt.xticks(fontsize=15)
# plt.xticks([0.004, 0.006, 0.010, 0.014])
plt.yticks(fontsize=20)
plt.xticks()
plt.savefig("fig/dpp1_tv.pdf")
plt.show()

Plotting empirical and exact probabilities

In [ ]:
# obtained by using a cutoff on sample size
proba_series = pd.Series(proba.values(), index=list(proba.keys()), name="true")
proba_estimated_series = pd.Series(proba_estimated.values(), index=list(proba_estimated.keys()), name="simulator")
proba_series.index.name = "outcome"
proba_estimated_series.index.name = "outcome"
df = pd.merge(proba_series, proba_estimated_series, on="outcome")
# df.plot.bar(figsize=(70,20))
df.plot.bar()

### Sampling the projection DPP by acceptance-rejection 
Comparing empirical acceptance probability and exact acceptance probability

In [ ]:
n = 10
rank = 5
np.random.seed(0) # fixing the seed for reproducibility
mat = np.random.randn(n,rank)
Q = mat / np.sqrt(np.sum(mat*mat, axis=0)) 
# Q, _ = np.linalg.qr(np.random.randn(n, n)) # for std prijection DPP # K = W @ W.T # the DPP kernel # only when Q comes from QR
W = Q[:,:rank]
# defining the circuit
circuit = ParallelCliffordLoader(W) # circuit = PyramidCliffordLoader(W)
# Add measurements
meas = qk.QuantumCircuit(n, n) # n qubits, n classical bits
meas.barrier(range(n)) # the barrier is optional, it is an instruction for the later transpiler
meas.measure(range(n), range(n)) # perform the measurement, record it in the classical bits
circuit.add_register(meas.cregs[0])
qc = circuit.compose(meas)
# simulation
num_shots = 10000
backend_sim = Aer.get_backend('qasm_simulator')
job_sim = backend_sim.run(
    qk.transpile(qc, backend_sim), 
    shots=num_shots
)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc)
# estimated success probability
nb_accepted_samples = 0
nb_rejected_samples = 0
for key in counts:
    sample = counts[key]
    spl_sz = utilities.plotting_utilities.sample_size(str(key))
    if spl_sz == rank:
        nb_accepted_samples += int(counts[key])
    else:
        nb_rejected_samples += int(counts[key])

print('emp success proba:', nb_accepted_samples/(nb_accepted_samples + nb_rejected_samples))
success_proba = np.linalg.det(W.T@W)
print('success proba:', success_proba)